# Chapter 5b

## SQL: Queries, Programming, Triggers

For this lecture, we will be using this Jupyter notebook to demonstrate queries.  We require the *ipython-sql* module, and the following lines load the module and create an in-memory database:

This is part b of the lecture, but we'll begin by loading up the same data as before.

In [1]:
%load_ext sql
%sql sqlite://

'Connected: None@None'

# Table Creation 

Now let's create some tables for sailors example and then add some data to them.  

We will create a table for:
* sailors
* boats
* reservations

*Note: I have added in a command to DROP the tables if they already exist.  This makes it easier to re-run these commands if necessary.*

In [2]:
%%sql 

DROP TABLE IF EXISTS sailors; 

CREATE TABLE sailors ( 
    sailor_id INTEGER PRIMARY KEY, 
    sailor_name varchar(20), 
    rating int, 
    age float);

Done.
Done.


[]

In [3]:
%%sql 

DROP TABLE IF EXISTS boats;

CREATE TABLE boats (
    boat_id INTEGER PRIMARY KEY,
    color VARCHAR(20)
);

Done.
Done.


[]

In [4]:
%%sql 
INSERT INTO BOATS (boat_id, color)
VALUES ( 101, 'red' );
INSERT INTO BOATS (boat_id, color)
VALUES ( 103, 'green');

1 rows affected.
1 rows affected.


[]

In [5]:
%%sql 

DROP TABLE IF EXISTS reservations;

CREATE TABLE reservations (
    sailor_id INTEGER, 
    boat_id INTEGER, 
    date int, 
    FOREIGN KEY(sailor_id) REFERENCES sailors(sailor_id));

Done.
Done.


[]

In [6]:
%%sql 
INSERT INTO SAILORS (sailor_id, sailor_name, rating, age)
VALUES ( 22, 'Dustin', 7, 45.0 );
INSERT INTO SAILORS (sailor_id, sailor_name, rating, age)
VALUES ( 31, 'Lubber', 8, 55.5 );
INSERT INTO SAILORS (sailor_id, sailor_name, rating, age)
VALUES ( 58, 'Rusty', 10, 35.0 );
INSERT INTO SAILORS (sailor_id, sailor_name, rating, age)
VALUES ( 28, 'Yuppy', 9, 45.0 );
INSERT INTO SAILORS (sailor_id, sailor_name, rating, age)
VALUES 
( 48, 'Guppy', 5, 35.0 ),
( 49, 'Slacky', 7, 41.5),
( 50, 'Happy', 7, 49);

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
3 rows affected.


[]

In [7]:
%%sql
INSERT INTO SAILORS (sailor_id, sailor_name, rating, age)
VALUES 
( 55, 'Yusuf', 10, 42.0 )

1 rows affected.


[]

In [8]:
%%sql
DELETE FROM reservations;


INSERT INTO RESERVATIONS (sailor_id, boat_id, date)
VALUES ( 22, 101, date('1996-10-10') );
INSERT INTO RESERVATIONS (sailor_id, boat_id, date)
VALUES ( 58, 103, date('1996-11-12') );
INSERT INTO RESERVATIONS (sailor_id, boat_id, date)
VALUES ( 58, 101, date('1996-11-15') );



0 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

# Review the Data
Let's take a look at all the data we just created:

In [9]:
%sql SELECT * from sailors;

Done.


sailor_id,sailor_name,rating,age
22,Dustin,7,45.0
28,Yuppy,9,45.0
31,Lubber,8,55.5
48,Guppy,5,35.0
49,Slacky,7,41.5
50,Happy,7,49.0
55,Yusuf,10,42.0
58,Rusty,10,35.0


In [10]:
%sql SELECT * from boats;

Done.


boat_id,color
101,red
103,green


In [11]:
%sql SELECT * from reservations;

Done.


sailor_id,boat_id,date
22,101,1996-10-10
58,103,1996-11-12
58,101,1996-11-15


# Aggregate Operators

Next we will examine aggregate operators like:
* COUNT (*)
* COUNT ( [DISTINCT] A)
* SUM ( [DISTINCT] A)
* AVG ( [DISTINCT] A)
* MAX (A)
* MIN (A)



In [26]:
%%sql 
SELECT COUNT (sailor_name) as sailor_count
FROM SAILORS;

Done.


sailor_count
8


In [27]:
%%sql 
SELECT AVG (S.age)
FROM SAILORS S
WHERE S.rating=10;

Done.


AVG (S.age)
38.5


In [14]:
%%sql 
SELECT AVG (S.age)
FROM SAILORS S
WHERE S.rating>7;

Done.


AVG (S.age)
44.375


In [30]:
%%sql
SELECT COUNT( DISTINCT S.rating)
FROM Sailors S;

Done.


COUNT( DISTINCT S.rating)
5


In [16]:
%%sql
SELECT MAX(S.rating) 
FROM Sailors S;

Done.


MAX(S.rating)
10


In [31]:
%%sql
SELECT S.sailor_name
FROM Sailors S
WHERE S.rating=(SELECT MAX(S2.rating) 
                FROM Sailors S2);

Done.


sailor_name
Yusuf
Rusty


In [32]:
%%sql 
SELECT AVG (S.age)
FROM Sailors S;

Done.


AVG (S.age)
43.5


# Find name and age of the oldest sailor(s)

In [19]:
%%sql 
SELECT S.sailor_name, MAX(S.age) 
FROM Sailors S;

Done.


sailor_name,MAX(S.age)
Lubber,55.5


In [20]:
%%sql 
SELECT S.sailor_name, S.age
FROM Sailors S
WHERE S.age=
    (SELECT MAX (S2.age)
    FROM Sailors S2);

Done.


sailor_name,age
Lubber,55.5


# Find name and age of the oldest sailor(s)

In [21]:
%%sql
SELECT S.sailor_name, S.age 
FROM Sailors S
WHERE (SELECT MAX(S2.age)
          FROM Sailors S2) = S.age;

Done.


sailor_name,age
Lubber,55.5


# Group By and Having

# Find the age of the youngest sailor with age >= 18, for reach rating with at least 2 such sailors

In [22]:
%%sql 
SELECT S.rating, MIN (S.age)
FROM Sailors S
WHERE S.age>=18
GROUP BY S.rating
HAVING COUNT(*)>1;

Done.


rating,MIN (S.age)
7,41.5
10,35.0


In [33]:
%%sql 
SELECT S.rating AS rating_group, MIN (S.age) AS minumum_age_of_rating
FROM Sailors S
WHERE S.age>=18
GROUP BY S.rating
HAVING COUNT(*)>1;

Done.


rating_group,minumum_age_of_rating
7,41.5
10,35.0


# For each red boat, find the number of reservations for this boat

In [34]:
%%sql 
SELECT b.color, b.boat_id, count(*) AS reservation_count
FROM Sailors S, Boats B, Reservations R
WHERE S.sailor_id=R.sailor_id 
    AND R.boat_id=B.boat_id 
    GROUP by B.boat_id, b.color;

Done.


color,boat_id,reservation_count
red,101,2
green,103,1
